In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT id, url, title, datePublished FROM articles \
                  WHERE datePublished<'2021-01-01' \
                  ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
!pip install torch
!pip install pytorch_pretrained_bert

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
def embed_text(text):
    # Text mit Start- und Endmarkern ergänzen
    marked_text = "[CLS] " + str(text) + " [SEP]"
    # Nur die ersten 512 Tokens verwenden
    tokenized = tokenizer.tokenize(marked_text)[0:512]
    # Und in IDs wandeln
    indexed = tokenizer.convert_tokens_to_ids(tokenized)
    # Der Einfachheit halber packst du alles in einen "Satz"
    segments_ids = [1] * len(tokenized)
    # Bisher hast du mit Listen gearbeitet, diese wandelst du jetzt in Tensoren
    tokens_tensor = torch.tensor([indexed])
    segments_tensors = torch.tensor([segments_ids])
    # Nun überträgst du die Token- und Segment-Tensoren in das Modell
    with torch.no_grad():
        # als Ergebnis erhältst du alle (!) Embedding-Layer
        el, _ = model(tokens_tensor, segments_tensors)
    return el

In [ ]:
from tqdm.auto import tqdm
tokenized_texts = []

sentence_embeddings = []

for text in tqdm(df["title"], total=len(df)):
    el = embed_text(text)
    token_vecs = el[11][0]
    embedding = torch.mean(token_vecs, dim=0)
    sentence_embeddings.append(embedding.numpy())

In [ ]:
text_bert = pd.DataFrame(sentence_embeddings)
text_bert.index = df.index

text_bert.to_sql("bert_articles", sql, index_label="id", if_exists="replace")

In [ ]:
et = embed_text("Microsoft schafft Umsatzsprung")

In [ ]:
token_vecs = et[11][0]
embedding = torch.mean(token_vecs, dim=0)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
r = cosine_similarity(sentence_embeddings, [embedding.numpy()])

In [ ]:
r.argmax()

In [ ]:
df.iloc[990]